In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import pandas as pd
import numpy as np
import zipfile
from sklearn.cluster import KMeans

In [ ]:
!curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1284k      0 --:--:-- --:--:-- --:--:-- 1283k


In [ ]:
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
  zip_ref.extractall('data')

In [ ]:
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [ ]:
print("movie dim", movies_df.shape)
print("rating dim", ratings_df.shape)

movie dim (9742, 3)
rating dim (100836, 4)


In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
num_users = len(ratings_df['userId'].unique())
num_movies = len(ratings_df['movieId'].unique())
print("num users", num_users)
print("num movies", num_movies)

num users 610
num movies 9724


In [ ]:
trainset = ratings_df.sample(frac=0.9, random_state=42).copy()
valset = ratings_df.drop(trainset.index).copy()

In [ ]:
trainset['rating'].values

array([4.5, 3. , 3. , ..., 3. , 5. , 1. ])

In [ ]:
valset.head()

,userId,movieId,rating,timestamp
2,1,6,4.0,964982224
28,1,527,5.0,964984002
55,1,1031,5.0,964982653
60,1,1073,5.0,964981680
66,1,1127,4.0,964982513


In [ ]:
for m in range (len(valset)):
  if valset.iloc[m]['rating'] > 3:
    print(valset.iloc[m])

Streaming output truncated to the last 5000 lines.
userId       5.280000e+02
movieId      2.571000e+03
rating       5.000000e+00
timestamp    1.391736e+09
Name: 83360, dtype: float64
userId       5.280000e+02
movieId      4.886000e+03
rating       3.500000e+00
timestamp    1.391737e+09
Name: 83369, dtype: float64
userId       5.280000e+02
movieId      4.851600e+04
rating       4.500000e+00
timestamp    1.391736e+09
Name: 83384, dtype: float64
userId       5.280000e+02
movieId      8.534200e+04
rating       5.000000e+00
timestamp    1.391736e+09
Name: 83400, dtype: float64
userId             530.0
movieId            592.0
rating               4.0
timestamp    843227185.0
Name: 83460, dtype: float64
userId       5.320000e+02
movieId      5.270000e+02
rating       5.000000e+00
timestamp    1.025570e+09
Name: 83494, dtype: float64
userId       5.320000e+02
movieId      2.329000e+03
rating       4.000000e+00
timestamp    1.025570e+09
Name: 83521, dtype: float64
userId       5.320000e+02
mov

In [ ]:
class MovieDataset(torch.utils.data.Dataset):
  def __init__(self, data):
    self.ratings = data.copy()

    users = data['userId'].unique()
    movies = data['movieId'].unique()


    self.useridx = {idx:userid for idx, userid in enumerate(users)}
    self.movieidx = {idx:movieid for idx, movieid in enumerate(movies)}


    self.userid = {i:o for o,i in self.useridx.items()}
    self.movieid = {i:o for o,i in self.movieidx.items()}


    self.ratings['userId'] = data['userId'].apply(lambda x: self.userid[x])
    self.ratings['movieId'] = data['movieId'].apply(lambda x: self.movieid[x])

    self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
    self.y = self.ratings['rating'].values
    self.x, self.y = torch.tensor(self.x), torch.tensor(self.y)

  def __getitem__(self, idx):
    return (self.x[idx], self.y[idx])

  def __len__(self):
    return len(self.ratings)





In [ ]:
class MFNet(nn.Module):
  def __init__(self, num_users, num_movies, embedding_dim=100, n_hidden=10):
    super(MFNet, self).__init__()
    self.user_embedding = nn.Embedding(num_users, embedding_dim)
    self.movie_embedding = nn.Embedding(num_movies, embedding_dim)
    # self.fc1 = nn.Linear(embedding_dim * 2, n_hidden)
    # self.fc2 = nn.Linear(n_hidden, 1)
    # self.dropout = nn.Dropout(0.1)
    self.user_embedding.weight.data.uniform_(0, 0.05)
    self.movie_embedding.weight.data.uniform_(0, 0.05)

  def forward(self, data):
    u, m = data[:, 0], data[:, 1]
    # U = self.user_embedding(u)
    # M = self.movie_embedding(m)
    # x = F.relu(torch.cat([U, M], dim=1))
    # x = self.dropout(x)
    # x = F.relu(self.fc1(x))
    # # x = self.dropout(x)
    # x = self.fc2(x)
    x = self.user_embedding(u) * self.movie_embedding(m)
    x = torch.sum(x, dim=1)
    return x

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(MLP, self).__init__()
        layers = []
        prev_dim = input_dim

        for h in hidden_dims:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            prev_dim = h

        layers.append(nn.Linear(prev_dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


In [ ]:
class TwoTowerModel(nn.Module):
    def __init__(
        self,
        user_input_dim,
        item_input_dim,
        embedding_dim=128,
        hidden_dims=(256, 128),
        normalize=True
    ):
        super(TwoTowerModel, self).__init__()

        self.user_tower = MLP(
            input_dim=user_input_dim,
            hidden_dims=hidden_dims,
            output_dim=embedding_dim
        )

        self.item_tower = MLP(
            input_dim=item_input_dim,
            hidden_dims=hidden_dims,
            output_dim=embedding_dim
        )

        self.normalize = normalize

    def encode_user(self, user_features):
        u = self.user_tower(user_features)
        return F.normalize(u, dim=-1) if self.normalize else u

    def encode_item(self, item_features):
        i = self.item_tower(item_features)
        return F.normalize(i, dim=-1) if self.normalize else i

    def forward(self, user_features, item_features):
        user_emb = self.encode_user(user_features)
        item_emb = self.encode_item(item_features)

        # similarity score
        scores = torch.sum(user_emb * item_emb, dim=-1)
        return scores

In [ ]:
def contrastive_loss(user_emb, item_emb, temperature=0.07):
    logits = user_emb @ item_emb.T
    logits /= temperature

    labels = torch.arange(user_emb.size(0), device=user_emb.device)
    loss = F.cross_entropy(logits, labels)
    return loss

In [ ]:

num_epochs = 20
lr = 0.001
wd = 0.0
embedding_dims = 100
n_hiddens = 10
loss_fn = nn.MSELoss()
model = MFNet(num_users, num_movies, embedding_dim=embedding_dims, n_hidden=n_hiddens)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

print(model)

train_data = MovieDataset(trainset)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)

val_data = MovieDataset(valset)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=True)


MFNet(
  (user_embedding): Embedding(610, 100)
  (movie_embedding): Embedding(9724, 100)
)


In [ ]:
model = TwoTowerModel(
    user_input_dim=64,
    item_input_dim=128,
    embedding_dim=128
).cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

model.train()

for batch in dataloader:
    user_features = batch["user_features"].cuda()
    item_features = batch["item_features"].cuda()

    user_emb = model.encode_user(user_features)
    item_emb = model.encode_item(item_features)

    loss = contrastive_loss(user_emb, item_emb)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def train(model, epochs):
  for epoch in range(num_epochs):
    model.train()
    losses = []
    for x, y in train_loader:
      x, y = x.cuda(), y.cuda()
      optimizer.zero_grad()
      outputs = model(x)
      loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      losses.append(loss.item())
      loss.backward()
      optimizer.step()
    print("epoch: ", epoch, "loss: ", np.mean(losses))


In [ ]:
def eval(model):
    model.eval()
    losses = []
    for x, y in val_loader:
      x, y = x.cuda(), y.cuda()
      outputs = model(x)
      print(outputs.squeeze())
      loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      losses.append(loss.item())
      print("loss: ", loss.item())
    print("test loss: ", np.mean(losses))

In [ ]:
from sklearn.metrics import precision_score, recall_score

def eval(model):
    model.eval()
    losses = []
    precision_scores = []
    recall_scores = []

    for x, y in val_loader:
      x, y = x.cuda(), y.cuda()
      outputs = model(x)
      # print(y.type(torch.float32))
      preds = outputs.squeeze().detach().cpu().numpy()
      targets = y.detach().cpu().numpy()

      p = precision_score(targets, preds, zero_division=0, average="weighted")
      r = recall_score(targets, preds, zero_division=0, average="weighted")

      # Append results
      precision_scores.append(p)
      recall_scores.append(r)


      # loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      # losses.append(loss.item())
      print("precision: ", precision_score)
      print("recall: ", recall_score)
    # print("test loss: ", np.mean(losses))
    print("test precision: ", np.mean(precision_scores))
    print("test recall: ", np.mean(recall_scores))

In [ ]:
train(model, num_epochs)

epoch:  0 loss:  5.489573775460925
epoch:  1 loss:  1.2730214674819171
epoch:  2 loss:  0.8989128241754217
epoch:  3 loss:  0.7871314206479803
epoch:  4 loss:  0.7422354699274044
epoch:  5 loss:  0.7202797267685824
epoch:  6 loss:  0.7060847028942807
epoch:  7 loss:  0.6928578849319679
epoch:  8 loss:  0.672586837129297
epoch:  9 loss:  0.6412727860116488
epoch:  10 loss:  0.5933394917564097
epoch:  11 loss:  0.5299225925810751
epoch:  12 loss:  0.4566600834701226
epoch:  13 loss:  0.38386429966084545
epoch:  14 loss:  0.3147496247535364
epoch:  15 loss:  0.2542250221555761
epoch:  16 loss:  0.20313441631487966
epoch:  17 loss:  0.16127255112353098
epoch:  18 loss:  0.12756155120216742
epoch:  19 loss:  0.10085673166283936


In [ ]:
eval(model)

In [ ]:
trained_movie_embeddings = model.movie_embedding.weight.data.cpu().numpy()
trained_movie_embeddings

array([[ 0.17761037, -0.00547321,  0.14373015, ...,  0.18426728,
         0.13860986, -0.0255557 ],
       [ 0.23371266,  0.14869596,  0.11028781, ...,  0.2495094 ,
         0.25965214,  0.18857417],
       [ 0.11208259, -0.07121215,  0.07276556, ...,  0.02477035,
         0.05558658,  0.12608081],
       ...,
       [ 0.02364905,  0.00777288,  0.01804801, ...,  0.00887611,
         0.00548178,  0.01182224],
       [ 0.03318582,  0.00210906,  0.03538558, ...,  0.03832636,
         0.04166934,  0.03239603],
       [ 0.00799335,  0.01968152,  0.01527428, ...,  0.00360363,
         0.04625057,  0.00652718]], dtype=float32)

In [ ]:
clusters = KMeans(n_clusters=10).fit_predict(trained_movie_embeddings)
for cluster in range(10):
  print("\nCluster #{}".format(cluster))
  for idx in np.where(clusters == cluster)[0][:10]:
    print(movies_df.iloc[idx]['title'])


Cluster #0
Sense and Sensibility (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Mighty Aphrodite (1995)
Dolores Claiborne (1995)
I.Q. (1994)
Losing Isaiah (1995)
Nell (1994)
Farewell My Concubine (Ba wang bie ji) (1993)
Firm, The (1993)
North (1994)

Cluster #1
Before and After (1996)
Higher Learning (1995)
Jurassic Park (1993)
Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996)
Operation Dumbo Drop (1995)
Sunset Park (1996)
Somebody to Love (1994)
Supercop 2 (Project S) (Chao ji ji hua) (1993)
Dante's Peak (1997)
Benny & Joon (1993)

Cluster #2
Cutthroat Island (1995)
Assassins (1995)
Lamerica (1994)
Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)
Misérables, Les (1995)
Vampire in Brooklyn (1995)
In the Bleak Midwinter (1995)
Braveheart (1995)
Chungking Express (Chung Hing sam lam) (1994)
Jupiter's Wife (1994)

Cluster #3
French Kiss (1995)
Three Colors: Red (Trois couleurs: Rouge) (1994)

Cluster #4
Four Rooms (1995)
Clueless (1995)
Postman,